In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
api_key = "5b2b73c6fb0f4202bd6e51132a657269"
base_url = "https://api.binance.com/api/v3/klines"

In [3]:
def to_milliseconds(date_str):
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    return int(dt.timestamp() * 1000)

def get_historical_data(symbol, start_date, end_date, interval='1d', limit=1000):
    start_timestamp = to_milliseconds(start_date)
    end_timestamp = to_milliseconds(end_date)
    url = f"{base_url}?symbol={symbol}&interval={interval}&startTime={start_timestamp}&endTime={end_timestamp}&limit={limit}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
    df["date"] = pd.to_datetime(df["timestamp"], unit='ms')
    df = df[["date", "open", "high", "low", "close", "volume"]]
    return df

In [4]:
btc_data = get_historical_data('BTCUSDT', '2014-01-01', '2024-01-01')

In [5]:
# проверка
print(btc_data.head())

# сохраняем все данные 
btc_data.to_csv('btc_data.csv', index=False)

        date           open           high            low          close  \
0 2017-08-17  4261.48000000  4485.39000000  4200.74000000  4285.08000000   
1 2017-08-18  4285.08000000  4371.52000000  3938.77000000  4108.37000000   
2 2017-08-19  4108.37000000  4184.69000000  3850.00000000  4139.98000000   
3 2017-08-20  4120.98000000  4211.08000000  4032.62000000  4086.29000000   
4 2017-08-21  4069.13000000  4119.62000000  3911.79000000  4016.00000000   

          volume  
0   795.15037700  
1  1199.88826400  
2   381.30976300  
3   467.08302200  
4   691.74306000  


In [ ]:
# годовые данные 
btc_data['year'] = btc_data['date'].dt.year
btc_yearly_data = btc_data.groupby('year')['close'].last()

In [ ]:
btc_yearly_data = pd.to_numeric(btc_yearly_data, errors='coerce')
btc_yearly_data = btc_yearly_data.dropna()
btc_returns = btc_yearly_data.pct_change() * 100  

# проверка 
print(f"Годовые доходности BTC:\n{btc_returns}")

Годовые доходности BTC:
year
2017          NaN
2018   -73.003771
2019    94.313376
2020    22.453209
Name: close, dtype: float64


In [11]:
# сохранение для BTC
btc_returns.to_csv("btc_returns.csv", header=True)

In [16]:
ada_data = get_historical_data('ADAUSDT', '2017-09-01', '2024-01-01')
ada_data["close"] = pd.to_numeric(ada_data["close"], errors='coerce')
ada_data = ada_data.dropna()
ada_yearly_data = ada_data.groupby(ada_data['date'].dt.year)['close'].last()  # Получаем цену на конец года
ada_returns = ada_yearly_data.pct_change() * 100

# сохранение ADA
ada_returns.to_csv("ada_returns.csv", header=True)

In [ ]:
btc_returns = btc_returns.dropna()
ada_returns = ada_returns.dropna()

# сохраняем результаты в CSV (мы удалили все NAN так как данные были собраны неправильно)
btc_returns.to_csv("btc_returns.csv", header=True)
ada_returns.to_csv("ada_returns.csv", header=True)

# проверка - года разные у обеих монет...
print("BTC Returns:\n", btc_returns)
print("ADA Returns:\n", ada_returns)

BTC Returns:
 year
2018   -73.003771
2019    94.313376
2020    22.453209
Name: close, dtype: float64
ADA Returns:
 date
2019    -18.968920
2020    452.024353
2021     66.278813
Name: close, dtype: float64


In [ ]:
combined_returns = pd.DataFrame({
    'BTC Returns': btc_returns,
    'ADA Returns': ada_returns
})

# убираем годы, которые есть только у одной из монет
combined_returns = combined_returns.dropna()

combined_returns.to_csv("combined_returns.csv", header=True)

print("Combined Returns:\n", combined_returns)

Combined Returns:
       BTC Returns  ADA Returns
2019    94.313376   -18.968920
2020    22.453209   452.024353
